<a href="https://colab.research.google.com/github/marquezjp/SQLToPandas/blob/main/SQLite/Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manipular SQLite com Pandas

## Preparar Ambiente

Para instalar o XlsxWriter

```
!pip install -U XlsxWriter
```

In [ ]:
# Carrega a Biblioteca do Oracle, YAML e Pandas
from datetime import date
import csv
import pandas as pd

In [ ]:
import xlsxwriter

In [ ]:
import sqlite3
from sqlalchemy.engine import create_engine

In [ ]:
today = date.today().strftime("%d/%m/%Y")
print("Hoje é", today)

## Conexão via SQL Alchemy

In [ ]:
# Cria a Conexão do SQLite
sqliteConfig = {
    'dialect': 'sqlite',
    'dbworkpath': 'Dados/SQLite',
    'dbfilename': 'sigrh',
    'dbext': 'sqlite'
}

dialeto = sqliteConfig.get('dialect', 'sqlite')

dbworkpath = sqliteConfig.get('dbworkpath', '')
dbfilename = sqliteConfig.get('dbfilename', '')
dbext = sqliteConfig.get('dbext', '')
dburl = f'{dbworkpath}/{dbfilename}.{dbext}' if dbfilename else ''

conexaoSQLite = create_engine(f'{dialeto}:///{dburl}', echo=False)
conexaoSQLite

## Executar Comandos SQLite

In [ ]:
# Excecutar comandos SQLite com conexão via SQL Alchemy
%%time
with conexaoSQLite.connect() as conexaoDb:
    for linha in conexaoDb.execute(f'SELECT name FROM sqlite_master WHERE type="table"'):
        print (linha)

## Executar a Consulta

In [ ]:
# Excecutar select do SQLite com conexão via Pandas e SQL Alchemy
%%time
base = pd.read_sql_query('select * from sigrh', conexaoSQLite)
base.head()

## Salva os Dados Selecionados no Arquivo CSV

In [ ]:
%%time
nomeArquivo = 'sigrh_schema'
base.to_csv(f'{nomeArquivo}.csv', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
%%time
nomeArquivo = 'sigrh_schema'
compression_opts = dict(method='zip', archive_name=f'{nomeArquivo}.csv')
base.to_csv(f'{nomeArquivo}.zip', index=False, encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC, compression=compression_opts)

## Salva os Dados Selecionados no Excel

In [ ]:
%%time
dataReferencia = date.today().strftime('%Y%m%d')
nomePlanilha = 'SCHEMA'
nomeArquivo = f'SIGRH-MCZ - {nomePlanilha} - {dataReferencia}.xlsx'

planilha = pd.ExcelWriter(nomeArquivo, engine='xlsxwriter', datetime_format='dd/mm/yy', date_format='dd/mm/yy')
base.to_excel(planilha, sheet_name=nomePlanilha, startrow=0, index=False)

# Formata Tabela
pagina = planilha.sheets[nomePlanilha]
linhas, colunas = base.shape
column_settings = [{'header': column} for column in base.columns.values.tolist()]
tabelaOpcoes = {
    'name': nomePlanilha,
    'style': 'Table Style Medium 11',
    'autofilter': False,
    'columns': column_settings
}
pagina.add_table(0, 0, linhas, colunas - 1, tabelaOpcoes)

# Formata Impressão
pagina.hide_gridlines(2)
pagina.freeze_panes(1, 0)
pagina.print_area(0,0,linhas, colunas)
pagina.repeat_rows(0,0)
pagina.fit_to_pages(1,0)
pagina.set_paper(9) # A4
pagina.set_portrait() # set_landscape or set_portrait()
#pagina.center_horizontally()
#pagina.center_vertically()
pagina.set_margins(left=0.7, right=0.7, top=1.14, bottom=0.75)

# Formata Cabecalho e Rodape
pagina.set_header('&R' + 'Data: &D')
pagina.set_footer('&R' +  'Página: &P of &N')

# Grava a Planilha
planilha.save()

In [ ]:
pd.read_excel(nomeArquivo, sheet_name=nomePlanilha, engine='openpyxl').head()

## Salva os Dados Selecionados no Google Sheet

In [ ]:
# Carregar as Bibliotecas e Prover a Autenticação no Google Drive
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
%%time
dataReferencia = date.today().strftime('%Y%m%d')
nomePlanilha = 'BASE'
nomeArquivo = f'SIGRH-MCZ - {nomePlanilha} - {dataReferencia}'

base = pd.read_sql(SQL, conexaoOracle)

# Criar a Planilha
planilha = gc.create(nomeArquivo)
planilha = gc.open(nomeArquivo)
pagina = municipios_arquivo.add_worksheet(nomePlanilha,rows=len(base),cols=len(base.columns))
pagina.update([base.columns.values.tolist()] + base.values.tolist())

## Salva os Dados Selecionados no SQLite

In [ ]:
%%time
nomeTabela = 'schema'
base.to_sql(name=nomeTabela, con=conexaoSQLite, if_exists="replace", index=False)

In [ ]:
%%time
pd.read_sql(f'select * from {nomeTabela}', conexaoSQLite).head()

## Salva os Dados de Arquivo CSV em partes no SQLite

In [ ]:
%%time
tamanhoPartes=10**5
nomeArquivo = 'nomeTabela'
nomeTabela = 'nomeTabela'
for partesArq in pd.read_csv(f'{nomeArquivo}.csv', encoding='utf-8', sep=';', decimal=',', quoting=csv.QUOTE_NONNUMERIC, chunksize=tamanhoPartes):
  partesArq.to_sql(name=nomeTabela, con=conexaoDB, if_exists="append", index=False)
  print(partesArq.iloc[0, 1])

In [ ]:
%%time
pd.read_sql(f'select count(*) from {nomeTabela}', conexaoSQLite).head()